In [1]:
import pandas as pd
import re 
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords, strip_short, stem_text
import pickle
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
# import en_core_web_sm
import nltk
import os
from nltk.tokenize import word_tokenize
import emoji

C:\Users\Annelie Schridde\AppData\Roaming\Python\Python38\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#import sys
#sys.path.append('../')
#import functions
#exec(open('./functions.py').read())
%run functions.py

[nltk_data] Downloading package stopwords to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Annelie
[nltk_data]     Schridde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.56 seconds)
Writing emoji data to C:\Users\Annelie Schridde\.demoji\codes.json ...
... OK


In [5]:
df = pickle.load(open('data_n_models/one_cleaned_df.pkl', "rb"))
de_df = df[df.language == 'de'].reset_index()
de_df.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language
0,0,1079317443523297280,12303,Boulder667,@spdde @LadyBitchRay1 @spdbt @amazonDE @Zaland...,0,"Liverpool, England",07-Mar-2021,116,False,0.0,1.368470e+18,zalando,de
1,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de
2,2,1193535074660171776,693,FabIan68286784,@missbellalugosi @Zalando Kleiner Einwand von ...,0,,06-Mar-2021,57,False,0.0,1.368196e+18,zalando,de
3,3,1318999115313876996,1554,mantis_spam,@missbellalugosi @Zalando Etwas aus PU als Jog...,0,,06-Mar-2021,1273,False,2.0,1.368196e+18,zalando,de
4,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de


In [6]:
def clean_complete(tweet):
    """
    tweet: pandas series
    prepares tweets complete cleaning for further lemmatization and dering embeddings
    """
    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #remove repeated charachters
    
    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    tweet = tweet.str.replace(r':-\)', ' smile')
    tweet = tweet.str.replace(r':-\(', ' sad')
    tweet = tweet.str.replace(r':-\/', ' confused')
    tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')
    tweet = tweet.str.replace('"', '')
    #to lower case
    tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [7]:
def clean_vader(tweet):
    """
    tweet: pandas series
    prepares tweets for vader sentiment analysis
    """

    pat = r"(\\n)|(@\w*)|((www\.[^\s]+)|(https?://[^\s]+))"
    tweet = tweet.str.replace(pat, '')

    #replace emoticons with words
    #SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

    #tweet = tweet.str.replace(r':-\)', ' smile')
    #tweet = tweet.str.replace(r':-\(', ' sad')
    #tweet = tweet.str.replace(r':-\/', ' confused')
    #tweet = tweet.str.replace(r':-P', ' playfullness')

    #delete \xa
    tweet = tweet.str.replace('\xa0', '')

    tweet = tweet.str.replace('&amp', '')
    tweet = tweet.str.replace('\n', '')

    #to lower case
    #tweet = tweet.str.lower()

    #covert hashtags to the normal text
    tweet = tweet.str.replace(r'#([^\s]+)', r'\1')

    #delete numbers
    tweet = [strip_numeric(c) for c in tweet]

    #replacing emojies with descriptions '❤️-> red heart'
    #tweet = [demoji.replace_with_desc(c, ' ') for c in tweet]

    #delete punctuation
    #tweet = [strip_punctuation(c) for c in tweet]

    #remove stop words
    #tweet = [remove_stopwords(c) for c in tweet]

    #remove short words
    tweet = [strip_short(c) for c in tweet]

    #remove mult whitespaces
    tweet = [strip_multiple_whitespaces(c) for c in tweet]
    return tweet


In [8]:
#import demoji
de_df['clean'] = clean_complete(de_df.tweet)
de_df.head()

<ipython-input-6-abe84fc2115d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
<ipython-input-6-abe84fc2115d>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\)', ' smile')
<ipython-input-6-abe84fc2115d>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\(', ' sad')
<ipython-input-6-abe84fc2115d>:16: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r':-\/', ' confused')
<ipython-input-6-abe84fc2115d>:29: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r'#([^\s]+)', r'\1')


,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean
0,0,1079317443523297280,12303,Boulder667,@spdde @LadyBitchRay1 @spdbt @amazonDE @Zaland...,0,"Liverpool, England",07-Mar-2021,116,False,0.0,1.368470e+18,zalando,de,quasi steuerfrei arbeiten
1,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...
2,2,1193535074660171776,693,FabIan68286784,@missbellalugosi @Zalando Kleiner Einwand von ...,0,,06-Mar-2021,57,False,0.0,1.368196e+18,zalando,de,kleiner einwand von mir zur getätigten aussage...
3,3,1318999115313876996,1554,mantis_spam,@missbellalugosi @Zalando Etwas aus PU als Jog...,0,,06-Mar-2021,1273,False,2.0,1.368196e+18,zalando,de,etwas aus als jogginghose bezeichnen ist minde...
4,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen playfullness


In [9]:
de_df['clean_vader'] = clean_vader(de_df.tweet)
de_df.head()

<ipython-input-7-dfb30a3f3187>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(pat, '')
<ipython-input-7-dfb30a3f3187>:28: FutureWarning: The default value of regex will change from True to False in a future version.
  tweet = tweet.str.replace(r'#([^\s]+)', r'\1')


,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader
0,0,1079317443523297280,12303,Boulder667,@spdde @LadyBitchRay1 @spdbt @amazonDE @Zaland...,0,"Liverpool, England",07-Mar-2021,116,False,0.0,1.368470e+18,zalando,de,quasi steuerfrei arbeiten,Quasi steuerfrei arbeiten
1,1,1079317443523297280,12303,Boulder667,"@spdde @LadyBitchRay1 Viel Spaß, damit eine Wa...",0,"Liverpool, England",07-Mar-2021,116,False,1.0,1.368148e+18,zalando,de,viel spaß damit eine wahl gewinnen stellt euch...,"Viel Spaß, damit eine Wahl gewinnen. Stellt Eu..."
2,2,1193535074660171776,693,FabIan68286784,@missbellalugosi @Zalando Kleiner Einwand von ...,0,,06-Mar-2021,57,False,0.0,1.368196e+18,zalando,de,kleiner einwand von mir zur getätigten aussage...,Kleiner Einwand von mir zur getätigten Aussage...
3,3,1318999115313876996,1554,mantis_spam,@missbellalugosi @Zalando Etwas aus PU als Jog...,0,,06-Mar-2021,1273,False,2.0,1.368196e+18,zalando,de,etwas aus als jogginghose bezeichnen ist minde...,Etwas aus als Jogginghose bezeichnen ist minde...
4,4,310342425,11476,AndyBaldauf,@dicecco @Zalando aber erst wenn sie wieder dü...,0,Thurgau | St. Gallen | Zürich,06-Mar-2021,1848,False,0.0,1.368146e+18,zalando,de,aber erst wenn sie wieder dürfen playfullness,aber erst wenn sie wieder dürfen :-P


In [10]:
print(de_df.tweet[2])
print(de_df.clean[2])
print(de_df.clean_vader[2])

@missbellalugosi @Zalando Kleiner Einwand von mir zur getätigten Aussage, auf dem Bild steht ja Oberstoff Material 100% Polyurethan und weiter unten Material 100% Latex. Kann es sein das nur die obere Schicht pu ist und die untere Latex oder ist das ein Denkfehler seitens Zalando?
kleiner einwand von mir zur getätigten aussage auf dem bild steht oberstoff material polyurethan und weiter unten material latex kann sein das nur die obere schicht ist und die untere latex oder ist das ein denkfehler seitens zalando
Kleiner Einwand von mir zur getätigten Aussage, auf dem Bild steht Oberstoff Material Polyurethan und weiter unten Material Latex. Kann sein das nur die obere Schicht ist und die untere Latex oder ist das ein Denkfehler seitens Zalando?


In [11]:
#Vater sentiment analyzer
#The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).

sid_obj = SentimentIntensityAnalyzer()
de_df['compound'] = [sid_obj.polarity_scores(c)['compound'] for c in de_df['clean_vader']]

In [12]:
print(de_df[['clean_vader', 'compound']].describe())

           compound
count  10222.000000
mean      -0.161686
std        0.368568
min       -0.966100
25%       -0.599400
50%        0.000000
75%        0.000000
max        0.969800


# Testing cleaning function

In [13]:
#testing
#delete links, users
pat = r"(\\n)|(@\w*)|((https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}))"
de_df.tweet = de_df.tweet.str.replace(pat, '')
de_df.tweet[2]

<ipython-input-13-5463527a300c>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(pat, '')


'  Kleiner Einwand von mir zur getätigten Aussage, auf dem Bild steht ja Oberstoff Material 100% Polyurethan und weiter unten Material 100% Latex. Kann es sein das nur die obere Schicht pu ist und die untere Latex oder ist das ein Denkfehler seitens Zalando?'

In [14]:
#remove chars more than twice
#str.replace( /(.)\1{2,}/g, '$1$1' )

In [15]:

#testing
# #replace emoticons with words
#SMILEYS = {":-(":"sad", ":‑)":"smiley", ":-P":"playfullness", ":-/":'confused'}

de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')
de_df.tweet = de_df.tweet.str.replace(r':-P', ' playfullness')

de_df.tweet[2]

<ipython-input-15-a5a60d69cb5d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\)', ' smile')
<ipython-input-15-a5a60d69cb5d>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\(', ' sad')
<ipython-input-15-a5a60d69cb5d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r':-\/', ' confused')


'  Kleiner Einwand von mir zur getätigten Aussage, auf dem Bild steht ja Oberstoff Material 100% Polyurethan und weiter unten Material 100% Latex. Kann es sein das nur die obere Schicht pu ist und die untere Latex oder ist das ein Denkfehler seitens Zalando?'

In [16]:
#testing#delete \xa
de_df.tweet = de_df.tweet.str.replace('\xa0', '')
de_df.tweet[0]

'     Quasi steuerfrei arbeiten'

In [17]:
#testing
de_df.tweet = de_df.tweet.str.replace('&amp', '')
de_df.tweet = de_df.tweet.str.replace('\n', '')
de_df.tweet[3140]


'Ich Befriedigte Mich Natürlich bereits Heute 🤤welch Dumme Frage.... Die Gewinnerin von 25 Euro Zalando ist Heute......'

In [18]:
#testing#to lower case
de_df.tweet = de_df.tweet.str.lower()
de_df.tweet[2]

'  kleiner einwand von mir zur getätigten aussage, auf dem bild steht ja oberstoff material 100% polyurethan und weiter unten material 100% latex. kann es sein das nur die obere schicht pu ist und die untere latex oder ist das ein denkfehler seitens zalando?'

In [19]:
#testing#covert hashtags to the normal text
de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')
de_df.tweet[2]

<ipython-input-19-46b7def794f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  de_df.tweet = de_df.tweet.str.replace(r'#([^\s]+)', r'\1')


'  kleiner einwand von mir zur getätigten aussage, auf dem bild steht ja oberstoff material 100% polyurethan und weiter unten material 100% latex. kann es sein das nur die obere schicht pu ist und die untere latex oder ist das ein denkfehler seitens zalando?'

In [20]:
#testing#delete numbers
de_df.tweet = [strip_numeric(c) for c in de_df.tweet]
de_df.tweet[7]

'können wir uns auf die bezeichnung „couchhose“ verständigen? fühle mich durch die formulierung „jogginghose“ mental unter druck gesetzt. muss das sein   ; co?'

In [22]:
#testing#replacing emojies with descriptions '❤️-> red heart'
de_df.tweet = [demoji.replace_with_desc(c, ' ') for c in de_df.tweet]
de_df.tweet[9]

'wir schließen eine partnerschaft mit  zur förderung von diversity ! mehr dazu in der pressemitteilung (link am ende)!//happy to announce our partnership with  to promote diversity! find more information in the press release:  '

In [23]:
##testing delete punctuation
de_df.tweet = [strip_punctuation(c) for c in de_df.tweet]
de_df.tweet[7]

'können wir uns auf die bezeichnung „couchhose“ verständigen  fühle mich durch die formulierung „jogginghose“ mental unter druck gesetzt  muss das sein     co '

In [24]:
#testing
de_df.tweet = [remove_stopwords(c) for c in de_df.tweet]
de_df.tweet[7]

'können wir uns auf die bezeichnung „couchhose“ verständigen fühle mich durch die formulierung „jogginghose“ mental unter druck gesetzt muss das sein'

In [25]:
#testin
de_df.tweet = [strip_multiple_whitespaces(c) for c in de_df.tweet]
de_df.tweet[7]

'können wir uns auf die bezeichnung „couchhose“ verständigen fühle mich durch die formulierung „jogginghose“ mental unter druck gesetzt muss das sein'

In [26]:
def lemmatize(tweet):
    '''
    tweet: pandas series
    should be applied on the cleaned tweets to transform words to their initial base form.
    For example: suggests -> suggest, deliveries -> delivery
    '''
    nlp = spacy.load("en_core_web_sm")
    tweet = [nlp(c) for c in tweet]
    tweet = [" ".join([token.lemma_ for token in t]) for t in tweet]
    return tweet

In [27]:
# import spacy
# spacy.load('en_core_web_sm')
# from spacy.lang.ge import German
# nlp = spacy.load("en_core_web_sm")
import spacy
nlp = spacy.load("de_core_news_sm")

In [72]:
de_df['lem'] = [nlp(c) for c in de_df.clean]

NameError: name 'nlp' is not defined

In [73]:
de_df.lem[868]

AttributeError: 'DataFrame' object has no attribute 'lem'

In [74]:
de_df['lemma'] = [" ".join([token.lemma_ for token in t]) for t in de_df.lem]
de_df.lemma[7]

AttributeError: 'DataFrame' object has no attribute 'lem'

In [75]:
de_df.head()

,level_0,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
0,0,0,793418126500000000,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN
1,1,1,1119231287000000128,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
2,2,2,1119231287000000128,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
3,3,3,1119231287000000128,23,pici1303,@Zalando Ich hab doch schon den Rücksendeschei...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
4,4,4,1119231287000000128,23,pici1303,@Zalando bei der Retoure ist noch ein einzelne...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN


# Data description
- tweet: initial tweet as it was downloaded from API
- clean: tweet cleaned completely from punctuation, emojies, emoticons, stopwords, special characters, users, hashtags, links
- clean_vader: tweet partly cleaned, saving punctuation, emoticons, emojies for using library Vader to get the sentiment of the tweet
- compouns: from -1 (negative) to 1 (positive), 0 - neutral, a tweet sentiment derived fro, Vader library
- lem: intermidiate step before lemmatization
- lemma: lemmatized words (sent -> send)

# Save the dataframe

In [97]:
pickle.dump(eng_df, open(r'.\de_df_labelled.pkl', 'wb'))